# TARDIS Configuration

* using the E6C geometry from libhkl
    * @cmazzoli found that, in this geometry, with the "lifting_detector_mu" mode, the following mapping applies:
    
| libhkl | TARDIS |
| :---:  | :---:  |
| mu     | theta  |
| gamma  | delta  |
| delta  | gamma  |
| phi    | None   |
| chi    | None   |
| omega  | None   |

* The diffractometer geometry with angle and axis definitions are depicted below

<img src="6c_diffractometer.png" width=480 height=320>
<img src="6c_angle_definitions.png" width=480 height=320>

* see [here](https://people.debian.org/~picca/hkl/diffractometers/e6c.html) for further documentation on libhkl

    

Begin by instantiating a calculation engine of the appropriate geometry, and configuring its mode as __lifting_detector_mu__ 

In [17]:
from hkl.calc import CalcE6C

tardis_calc = CalcE6C()

# what modes are available?
print('Available modes =', tardis_calc.engine.modes)
print('\nphysical axes =', tardis_calc.physical_axes)
print('\npseudo axes =', tardis_calc.pseudo_axes)

Available modes = ['bissector_vertical', 'constant_omega_vertical', 'constant_chi_vertical', 'constant_phi_vertical', 'lifting_detector_phi', 'lifting_detector_omega', 'lifting_detector_mu', 'double_diffraction_vertical', 'bissector_horizontal', 'double_diffraction_horizontal', 'psi_constant_vertical', 'psi_constant_horizontal', 'constant_mu_horizontal']

physical axes = OrderedDict([('mu', 0.0), ('omega', 0.0), ('chi', 0.0), ('phi', 0.0), ('gamma', 0.0), ('delta', 0.0)])

pseudo axes = OrderedDict([('h', 0.0), ('k', 0.0), ('l', 0.0)])


In [18]:
tardis_calc.engine.mode = 'lifting_detector_mu'

Next, seed the calculation engine with a parameterized sample and wavelength (or energy).

**NOTE**: length units are in Angstrom, angles are in degrees, and energy is in keV.

In [19]:
from hkl.util import Lattice

# lattice cell lengths are in Angstrom, angles are in degrees
lattice = Lattice(a=9.069, b=9.069, c=10.390, alpha=90.0, beta=90.0, gamma=120.0)
sample = tardis_calc.new_sample('sample1', lattice=lattice)

print(sample)

HklSample(name='sample1', lattice=LatticeTuple(a=9.069, b=9.069, c=10.39, alpha=90.0, beta=90.0, gamma=119.99999999999999), ux=Parameter(name='ux', limits=(-180.0, 180.0), value=0.0, fit=True, units='Degree'), uy=Parameter(name='uy', limits=(-180.0, 180.0), value=0.0, fit=True, units='Degree'), uz=Parameter(name='uz', limits=(-180.0, 180.0), value=0.0, fit=True, units='Degree'), U=array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]]), UB=array([[  7.99999720e-01,   3.99999860e-01,  -6.41365809e-17],
       [  0.00000000e+00,   6.92820080e-01,  -6.41365809e-17],
       [  0.00000000e+00,   0.00000000e+00,   6.04733908e-01]]), reflections=[], reflection_measured_angles=array([], shape=(0, 0), dtype=float64), reflection_theoretical_angles=array([], shape=(0, 0), dtype=float64))


In [20]:
tardis_calc.wavelength = 1.61198  # in Angstrom

# just to check
print('Energy =', tardis_calc.energy, 'keV')

Energy = 0.7691422970508319 keV


Now, apply constraints appropriate for TARDIS' geometry. This includes setting limits on the acceptable ranges of motion, initial (and constant!) values, and whether or not a particular axis should be factored into the fitting function that produces the forward and inverse solutions.

**NOTE**: physical motors should be checked that limits are in place prior to initiating any motion. Note also that none of the calculations below are associated with any physical motors, and that there is no connection between "limit" values used in the calculation, and soft-limit values that may be present in a control system for physical motors.

In [21]:
# Theta
mu = tardis_calc['mu']
mu.limits = (-181, 181)
mu.value = 0
mu.fit = True

# we don't have it. Fix to 0
phi = tardis_calc['phi']
phi.limits = (0, 0)
phi.value = 0
phi.fit = False

# we don't have it. Fix to 0
chi = tardis_calc['chi']
chi.limits = (0, 0)
chi.value = 0
chi.fit = False

# we don't have it!! Fix to 0
omega = tardis_calc['omega']
omega.limits = (0, 0)
omega.value = 0
omega.fit = False

# Attention naming convention inverted at the detector stages!
# delta
gamma = tardis_calc['gamma']
gamma.limits = (-5, 180)
gamma.value = 0
gamma.fit = True

# gamma
delta = tardis_calc['delta']
delta.limits = (-5, 180)
delta.value = 0
delta.fit = True

We can take a look at the UB matrix, but thus far, it won't be very interesting

In [22]:
tardis_calc.sample.UB

array([[  7.99999720e-01,   3.99999860e-01,  -6.41365809e-17],
       [  0.00000000e+00,   6.92820080e-01,  -6.41365809e-17],
       [  0.00000000e+00,   0.00000000e+00,   6.04733908e-01]])

Add two, known reflections and the motor positions associated with those hkl values.
Here, we are using values from @cmazolli's ESRF notes:

```
(3,3,0): del = 64.449, gam = -0.871, th = 25.285
(5,2,0): del = 79.712, gam = -1.374, th = 46.816
```

**NOTE**: the translation of gamma==delta, delta==gamma, and mu==theta is being used

In [23]:
r1 = tardis_calc.sample.add_reflection(3, 3, 0, 
                           position=tardis_calc.Position(gamma=64.449, mu=25.285, chi=0.0, phi=0.0, omega=0.0, delta=-0.871))

r2 = tardis_calc.sample.add_reflection(5, 2, 0,
                           position=tardis_calc.Position(gamma=79.712, mu=46.816, chi=0.0, phi=0.0, omega=0.0, delta=-1.374))

In [24]:
tardis_calc.sample.reflections

[(3.0, 3.0, 0.0), (5.0, 2.0, 0.0)]

Now a UB matrix can be computed.

In [25]:
tardis_calc.sample.compute_UB(r1, r2)

1

In [26]:
tardis_calc.sample.UB

array([[ 0.31323551, -0.4807593 ,  0.01113654],
       [ 0.73590724,  0.63942704,  0.01003773],
       [-0.01798898, -0.00176066,  0.60454803]])

Compare some libhkl-generated results with those from @cmazolli's notes:

```python
# Experimentally found reflections @ Lambda = 1.61198 A
# (4, 4, 0) = [90.628, 38.373, 0, 0, 0, -1.156]
# (4, 1, 0) = [56.100, 40.220, 0, 0, 0, -1.091]
# @ Lambda = 1.60911
# (6, 0, 0) = [75.900, 61.000, 0, 0, 0, -1.637]
# @ Lambda = 1.60954
# (3, 2, 0) = [53.090, 26.144, 0, 0, 0, -.933]
# (5, 4, 0) = [106.415, 49.900, 0, 0, 0, -1.535]
# (4, 5, 0) = [106.403, 42.586, 0, 0, 0, -1.183]
```

In [27]:
print(tardis_calc.forward((4,4,0)))

(PosCalcE6C(mu=38.37622128052063, omega=0.0, chi=0.0, phi=0.0, gamma=90.63030469353308, delta=-1.1613181970939916),)


In [28]:
print(tardis_calc.forward((4,1,0)))

(PosCalcE6C(mu=40.21991977757096, omega=0.0, chi=0.0, phi=0.0, gamma=56.09704093977082, delta=-1.083660865503293),)


Change wavelength here to 1.60911 Angstrom.
Note the difference below in `delta` (TARDIS' gamma axis)

In [29]:
# change wavelength
tardis_calc.wavelength = 1.60911
print(tardis_calc.forward((6,0,0)))

(PosCalcE6C(mu=60.99346591074179, omega=0.0, chi=0.0, phi=0.0, gamma=75.84521749189147, delta=-1.5839501607961701),)


In [30]:
tardis_calc.wavelength = 1.60954
print(tardis_calc.forward((3,2,0)))
print(tardis_calc.forward((5,4,0)))
print(tardis_calc.forward((4,5,0)))

(PosCalcE6C(mu=26.173823521308144, omega=0.0, chi=0.0, phi=0.0, gamma=53.05207622287554, delta=-0.8437995840438257),)
(PosCalcE6C(mu=49.892322604056034, omega=0.0, chi=0.0, phi=0.0, gamma=106.32053081067252, delta=-1.423656049079967),)
(PosCalcE6C(mu=42.54926633295045, omega=0.0, chi=0.0, phi=0.0, gamma=106.31894239326303, delta=-1.1854071532601609),)


# HKL PseudoPositioner Use

Let's explore the idea of an hkl 'motor'

In [33]:
from ophyd import Component as Cpt
from ophyd import (PseudoSingle, EpicsMotor)
from hkl.diffract import E6C


class Tardis(E6C):
    h = Cpt(PseudoSingle, '')
    k = Cpt(PseudoSingle, '')
    l = Cpt(PseudoSingle, '')
    
    theta = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X1}Mtr')
    omega = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X2}Mtr')
    chi = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X3}Mtr')
    phi = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X4}Mtr')
    delta = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X5}Mtr')
    gamma = Cpt(EpicsMotor, 'XF:31IDA-OP{Tbl-Ax:X6}Mtr')
    
# FIXME: hack to get around what should have been done at init of tardis_calc instance
tardis_calc._lock_engine = True

tardis = Tardis('', name='tardis', calc_inst=tardis_calc, energy=tardis_calc.energy)

# re-map Tardis' axis names onto what an E6C expects
name_map = {'mu': 'theta', 'omega': 'omega', 'chi': 'chi', 'phi': 'phi', 'gamma': 'delta', 'delta': 'gamma'}
tardis.calc.physical_axis_names = name_map

AssertionError: 

In [32]:
tardis.position

TypeError: Item 0: Must be number, not NoneType

In [ ]:
print('Energy =', tardis.energy, 'keV')

In [ ]:
tardis.move((1,0,1), wait=False)

In [ ]:
status = _

In [ ]:
status.done

In [ ]:
tardis.real_position

In [ ]:
tardis.position

In [ ]:
#tardis.set((1,0,2))

In [ ]:
tardis.h.describe()

In [ ]:
tardis.h.read()

In [ ]:
tardis.describe()

In [ ]:
tardis.read()

In [ ]:
tardis.position

In [ ]:
tardis.real_position